# Procesamiento de Lenguaje Natural

### Plan de la charla

#### Temático

* Modelos de embeddings de palabras
* Propiedades de los embeddings
* Traducción no-supervisada
* Traducción supervisada  

#### General

* Un viaje a través de distintas áreas de procesamiento de lenguaje natural y ciencia de datos en general

## Embeddings

* Nos referimos a *embeddings* cuando a cada objeto le asignamos un vector.
* En NLP/U, un objeto puede ser un documento, una frase o una palabra, por ejemplo
* La idea de usar vectores es conseguir alguna propiedad en ese espacio que nos dé información de los objetos;
 + por ejemplo, que 
 
 $$||\phi(\mathrm{Argentina}) - \phi(\mathrm{Brasil})|| > ||\phi(\mathrm{Argentina}) - \phi(\mathrm{Peretti})||$$
 
 
* Generalmente es una tarea no-supervisada.

### Embeddings de palabras  

* En este caso queremos asignarle a cada palabra un vector, de manera que palabras similares en semántica queden cercas en ese espacio.
* Como es una tarea no-supervisada, no tenemos explícitamente una referencia de las palabras similares que queremos que queden juntas.
* Definimos un objetivo parecido entonces: optimizar la predicción del contexto a partir de una palabra.
* Todos los modelos tienen generalmente un hiperparámetro $c$ que es la ventana de palabras a ver.
* Si dos palabras son semánticamente similares, pueden estar en contextos similares, por lo que deberían quedar cerca en el espacio.

#### La hipótesis distribucional
Conocerás una palabra por la compañía que lleva

#### Por qué esto  es una buena idea
Sea $\psi(p_1, p_2)$ la similaridad entre las palabras $p_1$ y $p_2$. 
* ¿Cómo se define la similaridad entre palabras? 
* ¿Cómo se define la similaridad entre vectores?

### Un modelo sencillo: la matriz de coocurrencias

* Como en esencia queremos codificar los contextos, podemos simplemente tener una matriz cuadrada donde cada fila y columna refiere a una palabra de nuestro vocabulario.
* Los elementos de la matriz son la cantidad de coocurrencias de la palabra $i$ con la palabra $j$.
* En definitiva, intentamos predecir la palabra actual en base al contexto (implícitamente).
* El modelo funciona más o menos bien pero la matriz se hace impracticable en *vocabularios* grandes:
    * La edición 2014 del DRAE tiene 93111 palabras
    * Eso serían alrededor de 17.35 GB

### Otros modelos basados en la matriz de coocurrencias
* Principalmente con el objetivo de reducir el tamaño de la matriz:
 + SVD (LSA)
 + otras factorizaciones (non-negative, etc.)
 + por proyecciones aleatorias

### CBOW y Skip-gram

* Modelos no supervisados

## CBOW
* La matriz de coocurrencias tiene el objetivo implícito de predecir la palabra a partir del contexto.
* Una forma explícita y reducida de hacer esto es usar el Continuous Bag of Words Model (CBOW)
![CBOW architecture](images/cbow_architecture.png)
* A partir de un *one-hot encoding* por $W$ obtenemos el embedding y luego aplicamos la otra capa de la red (softmax(W*h))
* El objetivo es predecir el one-hot encoding de $w(t)$ a partir de $w(t\pm j)$
* demo: https://ronxin.github.io/wevi/

## Skip-Gram

Si CBOW predice la palabra a partir del contexto, SG predice el contexto a partir de la palabra:
$$ \frac{1}{T} \sum_{t=1}^T \sum_{-c\leq j\leq c, j\neq 0} \log p(w_{t+j}|w_t)$$ 
![CBOW vs Skip Gram](images/cbow_vs_skipgram.png)
* Codifica explícitamente el orden de las palabras.

### FastText

* Se incluyen representaciones de n-gramas además de las palabras
    * Preservar el significado de palabras que son parte de otras
    * Capturar el significado de pre/posfijos

### Representación de los espacios: tSNE
* Algoritmo no lineal de reducción de dimensiones
Dado un espacio vectorial $D$ de gran dimensionalidad y un espacio de menor dimensionalidad $d$
* Puntos cercanos en $D$ quedan juntos en $d$ con gran probabilidad
* Puntos lejanos en $D$ quedan lejos en $d$ con gran probabilidad

## Propiedades de los embeddings

![LSA Dimensionality](images/LSAgraph.jpg)

* Pocos parámetros $\to$ underfitting
* Muchos parámetros $\to$ dimension como factor de regularización para la optimización 
* Va en contra del lema de Johnson-Lindenstrauss
    * n = d = 100000
    * fijamos $\epsilon$ = 0.8 (tasa de error alta)
    * k $\approx$ 308
    * A mayor $k$, menor $\epsilon$
        * En la práctica se observa _overfitting_ a mayor $k$

## Traducción no-supervisada

## Traducción supervisada

## Referencias
[1] Mikolov: Distributed Representations of Words and Phrases and their Compositionality https://arxiv.org/pdf/1310.4546.pdf   
[2] Mikolov: Efficient Estimation of Word Representations in
Vector Space https://arxiv.org/pdf/1301.3781.pdf  
[3] Rong: word2vec Parameter Learning Explained https://arxiv.org/pdf/1411.2738.pdf